<a href="https://colab.research.google.com/github/bhanu608/-_-/blob/main/Task3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Task 3 CAR PRICE PREDICTION WITH MACHINE LEARNING

In [15]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_log_error,mean_squared_error,mean_absolute_error,mean_absolute_percentage_error
from sklearn.ensemble import RandomForestRegressor


In [16]:
path = "https://raw.githubusercontent.com/bhanu608/-_-/main/carData.csv"
car_data = pd.read_csv(path)

In [17]:
car_data.head()

,car_ID,symboling,CarName,fueltype,aspiration,doornumber,carbody,drivewheel,enginelocation,wheelbase,...,enginesize,fuelsystem,boreratio,stroke,compressionratio,horsepower,peakrpm,citympg,highwaympg,price
0,1,3,alfa-romero giulia,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,13495.0
1,2,3,alfa-romero stelvio,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,16500.0
2,3,1,alfa-romero Quadrifoglio,gas,std,two,hatchback,rwd,front,94.5,...,152,mpfi,2.68,3.47,9.0,154,5000,19,26,16500.0
3,4,2,audi 100 ls,gas,std,four,sedan,fwd,front,99.8,...,109,mpfi,3.19,3.40,10.0,102,5500,24,30,13950.0
4,5,2,audi 100ls,gas,std,four,sedan,4wd,front,99.4,...,136,mpfi,3.19,3.40,8.0,115,5500,18,22,17450.0


In [18]:
#cleaning the data
car_data.info()
columns = ['CarName', 'fueltype', 'aspiration', 'doornumber', 'carbody', 'drivewheel', 'enginelocation', 'enginetype', 'cylindernumber', 'fuelsystem']
car_data.drop(columns, inplace=True, axis=1)
car_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 205 entries, 0 to 204
Data columns (total 26 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   car_ID            205 non-null    int64  
 1   symboling         205 non-null    int64  
 2   CarName           205 non-null    object 
 3   fueltype          205 non-null    object 
 4   aspiration        205 non-null    object 
 5   doornumber        205 non-null    object 
 6   carbody           205 non-null    object 
 7   drivewheel        205 non-null    object 
 8   enginelocation    205 non-null    object 
 9   wheelbase         205 non-null    float64
 10  carlength         205 non-null    float64
 11  carwidth          205 non-null    float64
 12  carheight         205 non-null    float64
 13  curbweight        205 non-null    int64  
 14  enginetype        205 non-null    object 
 15  cylindernumber    205 non-null    object 
 16  enginesize        205 non-null    int64  
 1

In [19]:
# data
X = car_data.iloc[:, :15]
y = car_data.iloc[:, 15]
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=25)

In [20]:
#training and testing the model
from sklearn.preprocessing import StandardScaler
model = RandomForestRegressor()
model.fit(X_train,y_train)
predictions=model.predict(X_test)
mse=mean_squared_error(y_test,predictions)
print("mean squared error : ",mse)

mean squared error :  3353878.8409859114


In [22]:
#testing on real time data
price = model.predict([[26, 1, 90.0, 150.0, 63.3 , 50.0, 1999, 20, 92, 3.0, 3.0, 10.0, 60, 5500, 30]])
print("The predicted price for the tuple ('CarName', 'fueltype', 'aspiration', 'doornumber', 'carbody', 'drivewheel', 'enginelocation', 'enginetype', 'cylindernumber', 'fuelsystem') : (26, 1, 90.0, 150.0, 63.3 , 50.0, 1999, 20, 92, 3.0, 3.0, 10.0, 60, 5500, 30) is : ",price)

The predicted price for the tuple ('CarName', 'fueltype', 'aspiration', 'doornumber', 'carbody', 'drivewheel', 'enginelocation', 'enginetype', 'cylindernumber', 'fuelsystem') : (26, 1, 90.0, 150.0, 63.3 , 50.0, 1999, 20, 92, 3.0, 3.0, 10.0, 60, 5500, 30) is :  [6092.615]


/usr/local/lib/python3.9/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
